In [1]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import numpy as np

from linearmodels.panel import PanelOLS
import datetime as dt

# Characteristcs OLS

Professor said we should start with a regular OLS Regression.
I quickly looked into this and think a random effects panel regression would probably fit our data better (still need to read more into this)
So probably we can first do a regular OLS and than compare it to the random effects panel regression. Just some initial thoughts though!!!
Here is some intial code to run them:


In [16]:
#decide which df to use
df = pd.read_csv("Dataframes/characteristics.csv")
#df = df.loc[:, ["Instrument", "Date", "Earnings Per Share - Actual Surprise", "Revenue - Actual", "Net Income after Tax"]]

df

,Instrument,Date,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Market Capitalization,loss firm status
0,POOL.OQ,2022-10-01,4.196,1.615339e+09,1.406160e+10,1.259813e+10,0
1,POOL.OQ,2022-07-01,1.503,2.055818e+09,1.556187e+10,1.405795e+10,0
2,POOL.OQ,2022-04-01,34.342,1.412650e+09,1.843682e+10,1.696712e+10,0
3,POOL.OQ,2022-01-01,40.267,1.035557e+09,2.384882e+10,2.268979e+10,0
4,POOL.OQ,2021-10-01,17.194,1.411448e+09,1.769954e+10,1.742020e+10,0
...,...,...,...,...,...,...,...
2485,AVY.N,2022-10-01,-0.131,2.317100e+09,1.622502e+10,1.322042e+10,0
2486,AVY.N,2022-07-01,11.635,2.347000e+09,1.650430e+10,1.343710e+10,0
2487,AVY.N,2022-04-01,10.595,2.349300e+09,1.753708e+10,1.441548e+10,0
2488,AVY.N,2022-01-01,0.294,2.183200e+09,2.087304e+10,1.793104e+10,0


#### OLS Regression

In [17]:
#identifying outliers and replacing them with NA
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7 #1.5 is standard threshold but we still want to keep enough variation in the data so setting threshol higher here
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_accuracy_new = df.copy()
df_accuracy_new["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_accuracy_new = df_accuracy_new.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_accuracy_new["Recommendation - Mean (1-5)"] = df_accuracy_new["Recommendation - Mean (1-5)"].fillna(0)
df_accuracy_new = df_accuracy_new.dropna()
df_accuracy_new.isna().sum()


Instrument                              0
Date                                    0
Earnings Per Share - Actual Surprise    0
Revenue - Actual                        0
Enterprise Value                        0
Market Capitalization                   0
loss firm status                        0
dtype: int64

In [18]:
df_accuracy_new

,Instrument,Date,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Market Capitalization,loss firm status
0,POOL.OQ,2022-10-01,4.196,1.615339e+09,1.406160e+10,1.259813e+10,0
1,POOL.OQ,2022-07-01,1.503,2.055818e+09,1.556187e+10,1.405795e+10,0
2,POOL.OQ,2022-04-01,34.342,1.412650e+09,1.843682e+10,1.696712e+10,0
3,POOL.OQ,2022-01-01,40.267,1.035557e+09,2.384882e+10,2.268979e+10,0
4,POOL.OQ,2021-10-01,17.194,1.411448e+09,1.769954e+10,1.742020e+10,0
...,...,...,...,...,...,...,...
2485,AVY.N,2022-10-01,-0.131,2.317100e+09,1.622502e+10,1.322042e+10,0
2486,AVY.N,2022-07-01,11.635,2.347000e+09,1.650430e+10,1.343710e+10,0
2487,AVY.N,2022-04-01,10.595,2.349300e+09,1.753708e+10,1.441548e+10,0
2488,AVY.N,2022-01-01,0.294,2.183200e+09,2.087304e+10,1.793104e+10,0


In [19]:
rescale = df_accuracy_new
#rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["Revenue - Actual"] = MinMaxScaler().fit_transform(np.array(rescale["Revenue - Actual"]).reshape(-1,1))
rescale["Enterprise Value"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value"]).reshape(-1,1))
rescale["Market Capitalization"] = MinMaxScaler().fit_transform(np.array(rescale["Market Capitalization"]).reshape(-1,1))

rescale


,Instrument,Date,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Market Capitalization,loss firm status
0,POOL.OQ,2022-10-01,4.196,0.010538,0.026228,0.002749,0
1,POOL.OQ,2022-07-01,1.503,0.013420,0.026725,0.003255,0
2,POOL.OQ,2022-04-01,34.342,0.009212,0.027679,0.004262,0
3,POOL.OQ,2022-01-01,40.267,0.006746,0.029473,0.006244,0
4,POOL.OQ,2021-10-01,17.194,0.009205,0.027434,0.004419,0
...,...,...,...,...,...,...,...
2485,AVY.N,2022-10-01,-0.131,0.015129,0.026945,0.002964,0
2486,AVY.N,2022-07-01,11.635,0.015325,0.027038,0.003040,0
2487,AVY.N,2022-04-01,10.595,0.015340,0.027380,0.003378,0
2488,AVY.N,2022-01-01,0.294,0.014253,0.028486,0.004596,0


In [20]:
y = rescale["Earnings Per Share - Actual Surprise"]
#X = df_accuracy_new[['Revenue - Actual', 'Net Income after Tax']]
X = rescale.iloc[:, 3:]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

model.summary()

C:\Users\alexa\mambaforge\envs\AnalystForecast\Lib\site-packages\statsmodels\regression\linear_model.py:1934: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


<class 'statsmodels.iolib.summary.Summary'>
"""
                                     OLS Regression Results                                     
================================================================================================
Dep. Variable:     Earnings Per Share - Actual Surprise   R-squared:                       0.003
Model:                                              OLS   Adj. R-squared:                  0.001
Method:                                   Least Squares   F-statistic:                     2.159
Date:                                  Fri, 24 Feb 2023   Prob (F-statistic):             0.0908
Time:                                          18:10:16   Log-Likelihood:                -10160.
No. Observations:                                  2405   AIC:                         2.033e+04
Df Residuals:                                      2401   BIC:                         2.035e+04
Df Model:                                             3                                         
Covariance Type:                              nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     5.1646      0.962      5.368      0.000       3.278       7.051
Revenue - Actual          6.0414      4.336      1.393      0.164      -2.462      14.545
Enterprise Value         59.3961     33.883      1.753      0.080      -7.046     125.838
Market Capitalization   -62.5866     31.874     -1.964      0.050    -125.089      -0.084
loss firm status               0          0        nan        nan           0           0
==============================================================================
Omnibus:                      416.522   Durbin-Watson:                   1.702
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3000.860
Skew:                           0.619   Prob(JB):                         0.00
Kurtosis:                       8.330   Cond. No.                          inf
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

#### Random Effects Panel Regression

In [21]:
df_accuracy_new['Date'] = pd.to_datetime(df_accuracy_new['Date'], infer_datetime_format=True)
df_accuracy_new.dtypes

Instrument                                      object
Date                                    datetime64[ns]
Earnings Per Share - Actual Surprise           float64
Revenue - Actual                               float64
Enterprise Value                               float64
Market Capitalization                          float64
loss firm status                                 int64
dtype: object

In [22]:
df_accuracy_new.set_index(["Instrument", "Date"], inplace=True)
df_accuracy_new

Earnings Per Share - Actual Surprise  Revenue - Actual  \
Instrument Date                                                                 
POOL.OQ    2022-10-01                                 4.196          0.010538   
           2022-07-01                                 1.503          0.013420   
           2022-04-01                                34.342          0.009212   
           2022-01-01                                40.267          0.006746   
           2021-10-01                                17.194          0.009205   
...                                                     ...               ...   
AVY.N      2022-10-01                                -0.131          0.015129   
           2022-07-01                                11.635          0.015325   
           2022-04-01                                10.595          0.015340   
           2022-01-01                                 0.294          0.014253   
           2021-10-01                                 4.784          0.013524   

                       Enterprise Value  Market Capitalization  \
Instrument Date                                                  
POOL.OQ    2022-10-01          0.026228               0.002749   
           2022-07-01          0.026725               0.003255   
           2022-04-01          0.027679               0.004262   
           2022-01-01          0.029473               0.006244   
           2021-10-01          0.027434               0.004419   
...                                 ...                    ...   
AVY.N      2022-10-01          0.026945               0.002964   
           2022-07-01          0.027038               0.003040   
           2022-04-01          0.027380               0.003378   
           2022-01-01          0.028486               0.004596   
           2021-10-01          0.028319               0.004404   

                       loss firm status  
Instrument Date                          
POOL.OQ    2022-10-01                 0  
           2022-07-01                 0  
           2022-04-01                 0  
           2022-01-01                 0  
           2021-10-01                 0  
...                                 ...  
AVY.N      2022-10-01                 0  
           2022-07-01                 0  
           2022-04-01                 0  
           2022-01-01                 0  
           2021-10-01                 0  

[2405 rows x 5 columns]

In [23]:
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
import statsmodels.graphics.tsaplots as tsap
from statsmodels.compat import lzip
from statsmodels.stats.diagnostic import het_white
from matplotlib import pyplot as plt

In [10]:
#decide which df to use
df = pd.read_csv("Dataframes/economic_story_dataframe.csv")
#df = df.loc[:, ["Instrument", "Date", "Earnings Per Share - Actual Surprise", "Revenue - Actual", "Net Income after Tax"]]

df

,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Earnings Per Share – Coefficient of Variation,Revenue - Actual,Enterprise Value,Number of Analysts,Recommendation - Mean (1-5),Recommendation - Mean (1-5).1,"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax,Price Target - Mean,Price Close
0,POOL.OQ,2022-10-01,4.78,4.58750,4.196,3.843706,1.615339e+09,1.406160e+10,10.0,2.27273,2.20000,3.747421,189965000.0,360.11111,318.21
1,POOL.OQ,2022-07-01,7.63,7.51700,1.503,3.625249,2.055818e+09,1.556187e+10,9.0,2.20000,2.00000,3.092855,307205000.0,435.37500,351.23
2,POOL.OQ,2022-04-01,4.23,3.14867,34.342,6.630736,1.412650e+09,1.843682e+10,9.0,2.00000,2.30000,2.325202,179203000.0,519.88889,422.85
3,POOL.OQ,2022-01-01,2.63,1.87500,40.267,3.269867,1.035557e+09,2.384882e+10,8.0,2.30000,2.30000,1.515266,107542000.0,571.00000,566.00
4,POOL.OQ,2021-10-01,4.51,3.84833,17.194,2.836295,1.411448e+09,1.769954e+10,8.0,2.30000,2.30000,1.527139,184573000.0,545.28571,434.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,AVY.N,2022-10-01,2.46,2.46323,-0.131,2.002655,2.317100e+09,1.622502e+10,13.0,2.14286,2.00000,3.747421,221500000.0,201.63636,162.70
2511,AVY.N,2022-07-01,2.64,2.36485,11.635,2.604816,2.347000e+09,1.650430e+10,13.0,2.00000,2.06667,3.092855,214500000.0,210.72727,161.87
2512,AVY.N,2022-04-01,2.40,2.17008,10.595,3.888797,2.349300e+09,1.753708e+10,12.0,2.06667,2.14286,2.325202,198200000.0,219.00000,173.97
2513,AVY.N,2022-01-01,2.13,2.12375,0.294,1.875927,2.183200e+09,2.087304e+10,12.0,2.14286,2.15385,1.515266,183100000.0,240.45455,216.57


In [11]:
#identifying outliers and replacing them with NA
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7 #1.5 is standard threshold but we still want to keep enough variation in the data so setting threshol higher here
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_accuracy_new = df.copy()
df_accuracy_new["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_accuracy_new = df_accuracy_new.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_accuracy_new["Recommendation - Mean (1-5)"] = df_accuracy_new["Recommendation - Mean (1-5)"].fillna(0)
df_accuracy_new = df_accuracy_new.dropna()
df_accuracy_new


,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Earnings Per Share – Coefficient of Variation,Revenue - Actual,Enterprise Value,Number of Analysts,Recommendation - Mean (1-5),Recommendation - Mean (1-5).1,"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax,Price Target - Mean,Price Close
0,POOL.OQ,2022-10-01,4.78,4.58750,4.196,3.843706,1.615339e+09,1.406160e+10,10.0,2.27273,2.20000,3.747421,189965000.0,360.11111,318.21
1,POOL.OQ,2022-07-01,7.63,7.51700,1.503,3.625249,2.055818e+09,1.556187e+10,9.0,2.20000,2.00000,3.092855,307205000.0,435.37500,351.23
2,POOL.OQ,2022-04-01,4.23,3.14867,34.342,6.630736,1.412650e+09,1.843682e+10,9.0,2.00000,2.30000,2.325202,179203000.0,519.88889,422.85
3,POOL.OQ,2022-01-01,2.63,1.87500,40.267,3.269867,1.035557e+09,2.384882e+10,8.0,2.30000,2.30000,1.515266,107542000.0,571.00000,566.00
4,POOL.OQ,2021-10-01,4.51,3.84833,17.194,2.836295,1.411448e+09,1.769954e+10,8.0,2.30000,2.30000,1.527139,184573000.0,545.28571,434.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,AVY.N,2022-10-01,2.46,2.46323,-0.131,2.002655,2.317100e+09,1.622502e+10,13.0,2.14286,2.00000,3.747421,221500000.0,201.63636,162.70
2511,AVY.N,2022-07-01,2.64,2.36485,11.635,2.604816,2.347000e+09,1.650430e+10,13.0,2.00000,2.06667,3.092855,214500000.0,210.72727,161.87
2512,AVY.N,2022-04-01,2.40,2.17008,10.595,3.888797,2.349300e+09,1.753708e+10,12.0,2.06667,2.14286,2.325202,198200000.0,219.00000,173.97
2513,AVY.N,2022-01-01,2.13,2.12375,0.294,1.875927,2.183200e+09,2.087304e+10,12.0,2.14286,2.15385,1.515266,183100000.0,240.45455,216.57


In [12]:
y_var_name = 'Earnings Per Share - Actual Surprise'
X_var_names = ['Revenue - Actual', "Net Income after Tax", "Number of Analysts", "Earnings Per Share – Coefficient of Variation", "Recommendation - Mean (1-5)", "WACC Inflation Adjusted Risk Free Rate, (%)", "Net Income after Tax", "Price Target - Mean", "Price Close"]
pooled_y=df_accuracy_new[y_var_name]
pooled_X=df_accuracy_new[X_var_names]

In [13]:
pooled_X = sm.add_constant(pooled_X)
pooled_X

,const,Revenue - Actual,Net Income after Tax,Number of Analysts,Earnings Per Share – Coefficient of Variation,Recommendation - Mean (1-5),"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax,Price Target - Mean,Price Close
0,1.0,1.615339e+09,189965000.0,10.0,3.843706,2.27273,3.747421,189965000.0,360.11111,318.21
1,1.0,2.055818e+09,307205000.0,9.0,3.625249,2.20000,3.092855,307205000.0,435.37500,351.23
2,1.0,1.412650e+09,179203000.0,9.0,6.630736,2.00000,2.325202,179203000.0,519.88889,422.85
3,1.0,1.035557e+09,107542000.0,8.0,3.269867,2.30000,1.515266,107542000.0,571.00000,566.00
4,1.0,1.411448e+09,184573000.0,8.0,2.836295,2.30000,1.527139,184573000.0,545.28571,434.41
...,...,...,...,...,...,...,...,...,...,...
2510,1.0,2.317100e+09,221500000.0,13.0,2.002655,2.14286,3.747421,221500000.0,201.63636,162.70
2511,1.0,2.347000e+09,214500000.0,13.0,2.604816,2.00000,3.092855,214500000.0,210.72727,161.87
2512,1.0,2.349300e+09,198200000.0,12.0,3.888797,2.06667,2.325202,198200000.0,219.00000,173.97
2513,1.0,2.183200e+09,183100000.0,12.0,1.875927,2.14286,1.515266,183100000.0,240.45455,216.57


In [14]:
pooled_olsr_model = sm.OLS(endog=pooled_y, exog=pooled_X)

In [15]:
pooled_olsr_model_results = pooled_olsr_model.fit()

In [16]:
print(pooled_olsr_model_results.summary())

                                     OLS Regression Results                                     
Dep. Variable:     Earnings Per Share - Actual Surprise   R-squared:                       0.048
Model:                                              OLS   Adj. R-squared:                  0.045
Method:                                   Least Squares   F-statistic:                     15.11
Date:                                  Fri, 24 Feb 2023   Prob (F-statistic):           8.22e-22
Time:                                          17:57:45   Log-Likelihood:                -10043.
No. Observations:                                  2394   AIC:                         2.010e+04
Df Residuals:                                      2385   BIC:                         2.016e+04
Df Model:                                             8                                         
Covariance Type:                              nonrobust                                         
                              

In [17]:
rescale = df_accuracy_new.iloc[:,5:]
rescale.drop("Number of Analysts", axis= 1)

for column in rescale:
    rescale[column] = MinMaxScaler().fit_transform(np.array(rescale[column]).reshape(-1,1))


rescale

,Earnings Per Share – Coefficient of Variation,Revenue - Actual,Enterprise Value,Number of Analysts,Recommendation - Mean (1-5),Recommendation - Mean (1-5).1,"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax,Price Target - Mean,Price Close
0,0.535296,0.010538,0.026228,0.214286,0.376963,0.343454,0.997960,0.523164,0.059242,0.052686
1,0.534579,0.013420,0.026725,0.190476,0.343454,0.251307,0.807848,0.524572,0.071909,0.058281
2,0.544436,0.009212,0.027679,0.190476,0.251307,0.389528,0.584891,0.523035,0.086132,0.070417
3,0.533414,0.006746,0.029473,0.166667,0.389528,0.389528,0.349653,0.522174,0.094734,0.094673
4,0.531992,0.009205,0.027434,0.166667,0.389528,0.389528,0.353101,0.523099,0.090406,0.072375
...,...,...,...,...,...,...,...,...,...,...
2510,0.529258,0.015129,0.026945,0.285714,0.317128,0.251307,0.997960,0.523543,0.032571,0.026335
2511,0.531233,0.015325,0.027038,0.285714,0.251307,0.282024,0.807848,0.523459,0.034101,0.026195
2512,0.535444,0.015340,0.027380,0.261905,0.282024,0.317128,0.584891,0.523263,0.035494,0.028245
2513,0.528842,0.014253,0.028486,0.261905,0.317128,0.322191,0.349653,0.523082,0.039104,0.035463


In [18]:
y = df_accuracy_new["Earnings Per Share - Actual Surprise"]
#X = df_accuracy_new[['Revenue - Actual', 'Net Income after Tax']]
X = rescale
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     OLS Regression Results                                     
================================================================================================
Dep. Variable:     Earnings Per Share - Actual Surprise   R-squared:                       0.051
Model:                                              OLS   Adj. R-squared:                  0.047
Method:                                   Least Squares   F-statistic:                     12.82
Date:                                  Fri, 24 Feb 2023   Prob (F-statistic):           4.60e-22
Time:                                          17:57:45   Log-Likelihood:                -10039.
No. Observations:                                  2394   AIC:                         2.010e+04
Df Residuals:                                      2383   BIC:                         2.016e+04
Df Model:                                            10                                         
Covariance Type:                              nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                           -47.3372      9.529     -4.968      0.000     -66.024     -28.651
Earnings Per Share – Coefficient of Variation    81.5192      8.961      9.097      0.000      63.948      99.091
Revenue - Actual                                  5.3992      4.264      1.266      0.206      -2.962      13.761
Enterprise Value                                -16.5602      7.653     -2.164      0.031     -31.568      -1.552
Number of Analysts                               -1.9995      2.369     -0.844      0.399      -6.644       2.645
Recommendation - Mean (1-5)                     -13.5237      6.819     -1.983      0.047     -26.896      -0.151
Recommendation - Mean (1-5).1                    11.0513      6.821      1.620      0.105      -2.325      24.428
WACC Inflation Adjusted Risk Free Rate, (%)      -7.2230      1.332     -5.424      0.000      -9.834      -4.612
Net Income after Tax                             31.5105     15.318      2.057      0.040       1.472      61.549
Price Target - Mean                              23.0174     49.324      0.467      0.641     -73.705     119.739
Price Close                                     -20.5408     56.364     -0.364      0.716    -131.068      89.986
==============================================================================
Omnibus:                      299.391   Durbin-Watson:                   1.733
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2801.829
Skew:                           0.213   Prob(JB):                         0.00
Kurtosis:                       8.283   Cond. No.                         355.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""